<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

In [12]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
outPath = "../data/vegFracControls.csv"

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [13]:
driver_paths = [os.path.join(dp, f) for (dp, _, fn) in os.walk("../data/") for f in fn if f.endswith('.nc')]
driver_names = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths]

file_table = pd.DataFrame({'filepath': driver_paths, 'file_name': driver_names})
file_table

,filepath,file_name
0,../data/BurntArea.nc,BurntArea
1,../data/crop.nc,crop
2,../data/Drought.nc,Drought
3,../data/MAP.nc,MAP
4,../data/MAT.nc,MAT
5,../data/MTWM.nc,MTWM
6,../data/pas.nc,pas
7,../data/PopDen.nc,PopDen
8,../data/SW1.nc,SW
9,../data/SW2.nc,SW


Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [14]:
from pdb import set_trace as browser
def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['layer'][:,:])
        gflat = gdata.flatten()
        #browser()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [15]:
values = [nc_extract(dp) for dp in driver_paths]

Processing: ../data/BurntArea.nc
Processing: ../data/crop.nc
Processing: ../data/Drought.nc
Processing: ../data/MAP.nc
Processing: ../data/MAT.nc
Processing: ../data/MTWM.nc
Processing: ../data/pas.nc
Processing: ../data/PopDen.nc
Processing: ../data/SW1.nc
Processing: ../data/SW2.nc
Processing: ../data/TreeCover.nc
Processing: ../data/urban.nc


Turn this into a dataframe for the analysis.

In [16]:
# turn list into a dataframe
fire_df = pd.DataFrame(np.array(values).T, columns=driver_names)

# replace null flags with pandas null
fire_df.replace(fire_df["BurntArea"][0], np.nan, inplace=True)

# drop all null rows (are ocean and not needed in optim)
fire_df.dropna(inplace=True)

Check that we've built it correctly.

In [17]:
fire_df.head()


,BurntArea,crop,Drought,MAP,MAT,MTWM,pas,PopDen,SW,SW,TreeCover,urban
128,0.000103,0.000000,0.503995,92.985718,18.424999,23.650000,0.004338,1.892931,355.882477,202.611023,0.023839,0.0
129,0.000038,0.000007,0.299137,111.214287,18.235714,24.842857,0.000955,2.009711,355.882477,210.107697,0.011756,0.0
130,0.000006,0.000002,0.360317,79.521431,19.132143,26.021429,0.001094,1.894917,355.882477,211.678619,0.004554,0.0
131,0.000008,0.000014,0.355147,58.299999,20.719048,27.778572,0.011957,0.497301,355.882477,214.360779,0.005536,0.0
132,0.000000,0.000000,0.259023,84.685715,21.566071,28.535715,0.014711,0.180588,355.882477,208.746124,0.019554,0.0


Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [18]:
savepath = os.path.expanduser(outPath)
fire_df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>